### **7.1 - Wait for multiple DAGs with the ExternalTaskSensor**

Dependencias DAG (DAG dependencies) es un tema tan importante, pero bastante difícil, porque la creación de dependencias entre sus tareas es una cosa, la creación de dependencias entre sus DAGs es otra cosa y puedes hacerlo aún más difícil que eso si empiezas a crear dependencias entre los deployments de Airflow. Pero esa es otra historia. En este vídeo, vamos a centrarnos en la primera forma de crear dependencias DAG utilizando el "ExternalTaskSensor". ¿Qué es el "ExternalTaskSensor"? Bueno, como puedes adivinar por su nombre, el "ExternalTaskSensor" es un sensor, por lo que espera a que algo suceda antes de pasar a la siguiente tarea, y en ese caso, este sensor esperará a que una tarea en otro DAG se complete antes de seguir adelante. Pongamos un ejemplo. Digamos que tienes el siguiente data pipeline "cleaning_dag" encargado de limpiar los XComs creados por otro DAG, en ese caso, "my_dag". Entonces, para hacer eso, necesitarás implementar el "ExternalTaskSensor" como la primera tarea de este DAG, que esperará a que la última tarea del otro DAG, en ese caso, "storing"

<center><img src="https://i.postimg.cc/1RDp88Rx/a1503.png"></center>
<center><img src="https://i.postimg.cc/SKwMshwN/a1504.png"></center>

para que tan pronto como "my_dag" termine, la siguiente tarea, "cleaning_xcoms", 

<center><img src="https://i.postimg.cc/6qK4rhWk/a1505.png"></center>

se dispara, y así tus XComs, serán eliminados. Veamos cómo hacerlo. En primer lugar, es necesario importar el "ExternalTaskSensor", así que vamos a hacer eso, escribe:

<center><img src="https://i.postimg.cc/5ymYnFj6/a1506.png"></center>

Una vez que lo tengas, crea una nueva tarea y llamémosla "waiting_for_task" igual a "ExternalTaskSensor" y escribe el task ID: 

<center><img src="https://i.postimg.cc/bY0tz4Lp/a1507.png"></center>

Y ahora necesitas especificar algunos argumentos aquí. El primero es "external_dag_id", que corresponde al DAG ID del DAG donde está la tarea que estás esperando. En este caso, es "my_dag". Luego tienes que especificar el task ID de la tarea que estás esperando, así que, en ese caso, es "storing". Y ya está. En este punto, has implementado con éxito la tarea "waiting_for_task" con el "ExternalTaskSensor". Así que vas a esperar a que se complete la tarea "storing" en el DAG "my_dag" antes de pasar a la siguiente tarea, "cleaning_xcoms". Veamos si funciona. Ve a la interfaz de usuario de Airflow, y como puedes ver tienes dos DAGs. 

<center><img src="https://i.postimg.cc/RZZfpG79/a1508.png"></center>

Ahora si haces click en "cleaning_dag" y vas a la Graph view, como puedes ver tienes el "ExternalTaskSensor". 

<center><img src="https://i.postimg.cc/43Kcsd2S/a1509.png"></center>
<center><img src="https://i.postimg.cc/YCDF5gRF/a1510.png"></center>

Vamos a activar el toggle y refrescar la página. Ahora el sensor está funcionando. Una cosa muy importante a tener en cuenta es que si pulsas sobre él y vas a login

<center><img src="https://i.postimg.cc/760TJRk1/a1511.png"></center>
<center><img src="https://i.postimg.cc/YqbLFgJP/a1512.png"></center>

Puedes ver que el sensor está esperando que la tarea "storing" en el DAG "my_dag" 

<center><img src="https://i.postimg.cc/wjMyQMzj/a1513.png"></center>

se complete en esta execution date

<center><img src="https://i.postimg.cc/zGQLtNDS/a1514.png"></center>

y es absolutamente importante recordar que el "ExternalTaskSensor" está esperando una tarea en otro DAG en una "execution date" específica, lo que significa que si en tu otro DAG, "storing" aquí, no se completa en esa execution date, tu sensor nunca tendrá éxito, y después de siete días "timeout". Ok, así que ten en cuenta esto. Ahora, volvamos a los DAGs y activemos el toggle de "my_dag" para ver qué pasa.

<center><img src="https://i.postimg.cc/j50DTh7D/a1515.png"></center>

Como puedes ver, se está ejecutando, esperemos un poco y podrás ver que tenemos la misma "execution date".

<center><img src="https://i.postimg.cc/G2FHmsFq/a1516.png"></center>

Ok, eso es lo que puedes ver aquí, así que eso significa que debería funcionar. Esperemos un poco, hasta que el DagRun se complete. Perfecto se ha completado. 

<center><img src="https://i.postimg.cc/jjJDB1LV/a1517.png"></center>

Vuelve a "cleaning_dag", luego a Graph View, y "waiting_for_task" sigue corriendo, haz click en el y puedes revisar los logs

<center><img src="https://i.postimg.cc/4yXn7NwZ/a1518.png"></center>
<center><img src="https://i.postimg.cc/5ypywdkz/a1519.png"></center>

y si vas a "Admin > XComs", puedes ver que tienes unos cuantos Xcoms aquí y el sensor funcionó como se esperaba. 

<center><img src="https://i.postimg.cc/15C4XSFh/a1520.png"></center>
<center><img src="https://i.postimg.cc/pLmrnSQ9/a1521.png"></center>

Eso es lo que se puede ver por el color del borde. Y lo que es más importante, "cleaning_xcoms" se ha ejecutado con éxito. Y si vuelves a "Admin > XComs", los XComs han sido eliminados.

<center><img src="https://i.postimg.cc/655yBqyf/a1522.png"></center>

Ok, recuerda que tus XComs no se eliminan automáticamente, así que, en realidad, estos DAG podrían ser super útiles para ti. Así es como puedes usar el "ExternalTaskSensor". Pero, hay algunos detalles que necesitas saber al respecto. Echémosles un vistazo. En primer lugar, ¿qué pasa si el DAG, donde se implementa el "ExternalTaskSensor", no tiene el mismo scheduling interval, así como la misma "start date" que el otro DAG donde está la tarea que estás esperando, lo que significa que no tendrás las mismas "execution dates". Bien, en ese caso puedes aprovechar otro argumento, que es "execution_delta", y este argumento espera un objeto timedelta. Básicamente, definirás el delta entre la "execution date" de ese DAG y la "execution dates" del otro DAG, donde está la tarea que estás esperando.

<center><img src="https://i.postimg.cc/Gm62PjzX/a1523.png"></center>

Además, si necesitas especificar el delta de una forma más compleja, entonces puedes usar otro argumento llamado "execution_date_fn", y esta vez, este argumento espera una función que tomará como argumento, la "fecha de ejecución actual (current execution date)", y entonces también puedes aprovechar el diccionario de "context" para que puedas pasar "ti", por ejemplo, u "dag object", lo que quieras como los otros argumentos de esa función. Lo que hay que recordar es que esta función devolverá las "execution dates", por lo que puedes tener múltiples "execution dates", en las que el "ExternalTaskSensor" comprobará la tarea que estás esperando en el otro DAG. Así que, ten en cuenta esto, porque ese argumento puede ser super útil para ti. 

<center><img src="https://i.postimg.cc/g0R0g6Tm/a1524.png"></center>

Muy bien, ahora, hay dos argumentos más que son bastante importantes con el "ExternalTaskSensor", y el primero es "failed_states", y este espera una lista de estados. La cosa es que, por defecto, está vacía, así que si la tarea que estás esperando falla entonces tu sensor seguirá funcionando hasta que se agote el tiempo de espera (timeout), así que por eso tienes que definir los "failed_states". Los estados en los que dices, ok, si la tarea que estoy esperando ha fallado, o se ha saltado, entonces quiero poner el sensor en fallo (failure). Así que, en ese caso, puedes especificar "failed" y por qué no "skipped".

<center><img src="https://i.postimg.cc/9MhQKF6T/a1525.png"></center>

Entonces el último argumento, que puede que necesites definir es "allowed_states" y en ese caso, de nuevo, necesitas especificar tu lista, y aquí puedes decir "success", lo que significa que si la tarea que estás esperando tuvo éxito, entonces funciona. 

<center><img src="https://i.postimg.cc/RhCVCpVZ/a1526.png"></center>

Muy bien, así es como funciona el "ExternalTaskSensor". No dudes en jugar con él, porque puede ser bastante complejo de usar al principio y ten en cuenta que lo bueno con el "ExternalTaskSensor", es que puedes crear múltiples dependencias upstream entre el DAG, donde se implementarán los "ExternalTaskSensor" y tus otros DAGs. Dicho esto, vamos a descubrir la otra forma de crear dependencias DAG, esta vez con el "TriggerDagRunOperator".